# * VINSIGHT : Inflow
    Prepaid(GEO Channel) : DTAC รอพี่หมอ re-run ต้องไม่นับ Wholesales, AE

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R00010001CS' --Total Inflow M1
        , 'TB0R00010001CS' --Total Inflow M1 : TRUE
        , 'DB0R00010001CS' --Total Inflow M1 : DTAC
        , 'B0R00010001CG' --Total Inflow M1 - GEO Channel
        , 'TB0R00010001CG' --Total Inflow M1 : TRUE - GEO Channel
        , 'DB0R00010001CG' --Total Inflow M1 : DTAC - GEO Channel
        
        , 'B1R000900CS' --Prepaid Inflow M1
        , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
        , 'DB1R000900CS' --Prepaid Inflow M1 : DTAC
        , 'B1R000900CG' --Prepaid Inflow M1 - GEO Channel
        , 'TB1R000900CG' --Prepaid Inflow M1 : TMH - GEO Channel
        , 'DB1R000900CG' --Prepaid Inflow M1 : DTAC - GEO Channel
        
        , 'B2R010500CS' --Postpaid Inflow M1 B2C
        , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
        , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
        , 'B2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel
        , 'TB2R010500CG' --Postpaid Inflow M1 B2C : TMH - GEO Channel
        , 'DB2R010500CG' --Postpaid Inflow M1 B2C : DTAC - GEO Channel
        , 'B2R020500CS' --Postpaid Inflow M1 B2B
        , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
        , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
        , 'B2R020500CG' --Postpaid Inflow M1 B2B - GEO Channel
        , 'DB2R020500CG' --Postpaid Inflow M1 B2B : DTAC - GEO Channel
        , 'TB2R020500CG' --Postpaid Inflow M1 B2B : TMH - GEO Channel
        
        , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
        , 'TB3R000601CG' --TOL Inflow M1 Connected : Consumer - GEO Channel
        , 'TB3R000601D1CS' --TOL Inflow M1 Connected : Consumer (Install Location)
        , 'TB3R000601D1CG' --TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
        , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
        , 'TB3R000602CG' --TOL Inflow M1 Connected : DataService - GEO Channel
        , 'TB3R000602D1CS' --TOL Inflow M1 Connected : DataService (Install Location)
        , 'TB3R000602D1CG' --TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

        , 'TB4R001700CS' --TVS Now Inflow M1
        , 'TB4R001700CG' --TVS Now Inflow M1 - GEO Channel
        , 'TB4R001004CS' --TVS CMDU Inflow M1
        , 'TB4R001004CG' --TVS CMDU Inflow M1 - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-06-26, 10:17:00

DataFrame: 6336 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

C:\Users\Narut4\AppData\Local\Temp\ipykernel_21232\1881320084.py:4: RuntimeWarning: invalid value encountered in cast
  prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)


## Overview(All) by Period

In [4]:
''' Inflow M1(All Channel) Yearly '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_yearly_df = m1_yearly_df.loc[m1_yearly_df['METRIC_CD'].isin(v_metric_list)]

m1_yearly_df['TOTAL'] = np.where(m1_yearly_df['METRIC_CD']=='B0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOTAL_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOTAL_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE'] = np.where(m1_yearly_df['METRIC_CD']=='B1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C'] = np.where(m1_yearly_df['METRIC_CD']=='B2R010500CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB2R010500CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB2R010500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B'] = np.where(m1_yearly_df['METRIC_CD']=='B2R020500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB2R020500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB2R020500CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOL_CONN'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000601CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_CONN(Install)'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000601D1CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_DSV'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000602CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_DSV(Install)'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000602D1CS', m1_yearly_df['P'], 0)
m1_yearly_df['TVS_NOW'] = np.where(m1_yearly_df['METRIC_CD']=='TB4R001700CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TVS_CMDU'] = np.where(m1_yearly_df['METRIC_CD']=='TB4R001004CS', m1_yearly_df['P'], 0)

# m1_yearly_df = m1_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_yearly_df['YoY'] = m1_yearly_df['TOTAL'] - m1_yearly_df['TOTAL'].shift()
# m1_yearly_df = m1_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# m1_yearly_df = m1_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_yearly_df = m1_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_yearly_df['YoY'] = m1_yearly_df['TOTAL'] - m1_yearly_df['TOTAL'].shift()
m1_yearly_df = m1_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
m1_yearly_df = m1_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_yearly_df_display = m1_yearly_df.copy()
m1_yearly_df_display['ACTUAL_AS_OF'] = m1_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_yearly_df_display[col] = m1_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025,2025-06-26 01:10:35,20250624,0,"3,205,699,541","1,694,681,482","1,511,018,059","2,432,772,647","1,121,090,465","1,318,117,241","476,632,057","369,915,321","106,716,736","156,789,621","2,197,463"


In [5]:
''' Inflow M1(All Channel) Quarterly '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_quarterly_df = m1_quarterly_df.loc[m1_quarterly_df['METRIC_CD'].isin(v_metric_list)]

m1_quarterly_df['TOTAL'] = np.where(m1_quarterly_df['METRIC_CD']=='B0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOTAL_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOTAL_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE'] = np.where(m1_quarterly_df['METRIC_CD']=='B1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C'] = np.where(m1_quarterly_df['METRIC_CD']=='B2R010500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB2R010500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB2R010500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B'] = np.where(m1_quarterly_df['METRIC_CD']=='B2R020500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB2R020500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB2R020500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOL_CONN'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000601CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_CONN(Install)'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000601D1CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_DSV'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000602CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_DSV(Install)'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000602D1CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TVS_NOW'] = np.where(m1_quarterly_df['METRIC_CD']=='TB4R001700CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TVS_CMDU'] = np.where(m1_quarterly_df['METRIC_CD']=='TB4R001004CS', m1_quarterly_df['P'], 0)

# m1_quarterly_df = m1_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_quarterly_df['QoQ'] = m1_quarterly_df['TOTAL'] - m1_quarterly_df['TOTAL'].shift()
# m1_quarterly_df = m1_quarterly_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# m1_quarterly_df = m1_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_quarterly_df = m1_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_quarterly_df['QoQ'] = m1_quarterly_df['TOTAL'] - m1_quarterly_df['TOTAL'].shift()
m1_quarterly_df = m1_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
m1_quarterly_df = m1_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_quarterly_df_display = m1_quarterly_df.copy()
m1_quarterly_df_display['ACTUAL_AS_OF'] = m1_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_quarterly_df_display[col] = m1_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,20251,2025-06-26 01:10:35,20250331,0,"1,778,578,561","918,417,080","860,161,482","1,379,322,046","619,695,416","759,626,629","249,140,023","195,429,182","53,710,841","79,484,928","1,120,189"
1,20252,2025-06-26 01:10:35,20250624,"-351,457,582","1,427,120,979","776,264,402","650,856,578","1,053,450,602","501,395,049","558,490,612","227,492,033","174,486,138","53,005,895","77,304,693","1,077,274"


In [6]:
''' Inflow M1(All Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
m1_monthly_df = m1_monthly_df.loc[m1_monthly_df['METRIC_CD'].isin(v_metric_list)]

m1_monthly_df['TOTAL'] = np.where(m1_monthly_df['METRIC_CD']=='B0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOTAL_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOTAL_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE'] = np.where(m1_monthly_df['METRIC_CD']=='B1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C'] = np.where(m1_monthly_df['METRIC_CD']=='B2R010500CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB2R010500CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB2R010500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B'] = np.where(m1_monthly_df['METRIC_CD']=='B2R020500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB2R020500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB2R020500CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOL_CONN'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000601CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_CONN(Install)'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000601D1CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_DSV'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000602CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_DSV(Install)'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000602D1CS', m1_monthly_df['P'], 0)
m1_monthly_df['TVS_NOW'] = np.where(m1_monthly_df['METRIC_CD']=='TB4R001700CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TVS_CMDU'] = np.where(m1_monthly_df['METRIC_CD']=='TB4R001004CS', m1_monthly_df['P'], 0)

# m1_monthly_df = m1_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_monthly_df['MoM'] = m1_monthly_df['TOTAL'] - m1_monthly_df['TOTAL'].shift()
# m1_monthly_df = m1_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# m1_monthly_df = m1_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_monthly_df = m1_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_monthly_df['MoM'] = m1_monthly_df['TOTAL'] - m1_monthly_df['TOTAL'].shift()
m1_monthly_df = m1_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
m1_monthly_df = m1_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_monthly_df_display = m1_monthly_df.copy()
m1_monthly_df_display['ACTUAL_AS_OF'] = m1_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_monthly_df_display[col] = m1_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202501,2025-06-26 01:10:35,20250131,0,"639,880,599","319,922,636","319,957,964","508,715,782","225,749,189","282,966,592","79,460,514","61,334,089","18,126,426","25,646,982","382,335"
1,202502,2025-06-26 01:10:35,20250228,"-64,700,198","575,180,401","288,071,249","287,109,152","442,103,679","192,368,192","249,735,487","81,132,607","63,185,463","17,947,144","25,046,745","267,183"
2,202503,2025-06-26 01:10:35,20250331,"-11,662,840","563,517,561","310,423,195","253,094,366","428,502,585","201,578,035","226,924,550","88,546,902","70,909,630","17,637,272","28,791,201","470,671"
3,202504,2025-06-26 01:10:35,20250430,"-29,481,414","534,036,147","295,795,804","238,240,343","395,842,378","191,822,637","204,019,741","87,789,592","69,118,915","18,670,677","26,236,110","441,580"
4,202505,2025-06-26 01:10:35,20250531,"-1,532,940","532,503,207","287,111,076","245,392,131","400,164,161","190,514,414","209,649,746","79,167,109","59,723,743","19,443,366","28,292,314","401,782"
5,202506,2025-06-26 01:10:35,20250624,"-171,921,581","360,581,626","193,357,522","167,224,104","257,444,063","119,057,998","144,821,125","60,535,332","45,643,480","14,891,852","22,776,269","233,913"


In [7]:
''' Inflow M1(All Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_weekly_df = m1_weekly_df.loc[m1_weekly_df['METRIC_CD'].isin(v_metric_list)]

m1_weekly_df['TOTAL'] = np.where(m1_weekly_df['METRIC_CD']=='B0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOTAL_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOTAL_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE'] = np.where(m1_weekly_df['METRIC_CD']=='B1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C'] = np.where(m1_weekly_df['METRIC_CD']=='B2R010500CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB2R010500CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB2R010500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B'] = np.where(m1_weekly_df['METRIC_CD']=='B2R020500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB2R020500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB2R020500CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOL_CONN'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000601CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_CONN(Install)'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000601D1CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_DSV'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000602CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_DSV(Install)'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000602D1CS', m1_weekly_df['P'], 0)
m1_weekly_df['TVS_NOW'] = np.where(m1_weekly_df['METRIC_CD']=='TB4R001700CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TVS_CMDU'] = np.where(m1_weekly_df['METRIC_CD']=='TB4R001004CS', m1_weekly_df['P'], 0)

# m1_weekly_df = m1_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_weekly_df['WoW'] = m1_weekly_df['TOTAL'] - m1_weekly_df['TOTAL'].shift()
# m1_weekly_df = m1_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# m1_weekly_df = m1_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_weekly_df = m1_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_weekly_df['WoW'] = m1_weekly_df['TOTAL'] - m1_weekly_df['TOTAL'].shift()
m1_weekly_df = m1_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
m1_weekly_df = m1_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_weekly_df_display = m1_weekly_df.copy()
m1_weekly_df_display['ACTUAL_AS_OF'] = m1_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_weekly_df_display[col] = m1_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025018,2025-06-26 01:10:35,20250504,0,"71,297,075","38,313,373","32,983,702","51,937,498","24,423,793","27,513,705","12,596,056","10,018,196","2,577,860","3,113,313","63,934"
1,2025019,2025-06-26 01:10:35,20250511,"47,225,508","118,522,584","67,337,840","51,184,744","87,456,449","43,643,248","43,813,201","19,633,666","14,975,619","4,658,047","7,009,281","72,921"
2,2025020,2025-06-26 01:10:35,20250518,"717,446","119,240,030","68,012,343","51,227,687","89,823,692","45,955,559","43,868,133","18,191,170","13,807,640","4,383,529","6,545,820","55,989"
3,2025021,2025-06-26 01:10:35,20250525,"-6,070,828","113,169,202","58,733,279","54,435,923","88,667,894","39,415,512","49,252,382","15,449,068","10,996,556","4,452,512","6,365,317","99,707"
4,2025022,2025-06-26 01:10:35,20250601,"13,061,805","126,231,007","63,074,661","63,156,346","94,336,319","42,120,000","52,216,319","16,127,590","12,235,090","3,892,500","6,146,633","122,633"
5,2025023,2025-06-26 01:10:35,20250608,"-17,312,723","108,918,285","58,145,779","50,772,505","80,639,520","35,669,535","44,969,985","18,709,895","14,415,759","4,294,136","6,644,492","78,851"
6,2025024,2025-06-26 01:10:35,20250615,"-2,233,494","106,684,790","59,701,196","46,983,594","80,227,694","38,816,433","41,411,261","17,171,931","13,204,340","3,967,591","6,227,611","56,245"
7,2025025,2025-06-26 01:10:35,20250622,"8,014,086","114,698,876","60,709,763","53,989,113","84,519,158","39,528,333","44,990,825","16,799,804","11,900,021","4,899,783","6,986,401","75,397"
8,2025026,2025-06-26 01:10:35,20250624,"-100,375,894","14,322,983","6,440,362","7,882,620",0,0,"6,435,059","5,023,262","3,814,002","1,209,260","2,029,715","10,018"


In [8]:
''' Inflow M1(All Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_daily_df = m1_daily_df.loc[m1_daily_df['METRIC_CD'].isin(v_metric_list)]

m1_daily_df['TOTAL'] = np.where(m1_daily_df['METRIC_CD']=='B0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_T'] = np.where(m1_daily_df['METRIC_CD']=='TB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_D'] = np.where(m1_daily_df['METRIC_CD']=='DB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['PRE'] = np.where(m1_daily_df['METRIC_CD']=='B1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_T'] = np.where(m1_daily_df['METRIC_CD']=='TB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_D'] = np.where(m1_daily_df['METRIC_CD']=='DB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C'] = np.where(m1_daily_df['METRIC_CD']=='B2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R010500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B'] = np.where(m1_daily_df['METRIC_CD']=='B2R020500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R020500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R020500CS', m1_daily_df['P'], 0)
m1_daily_df['TOL_CONN'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_CONN(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601D1CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_DSV'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000602CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_DSV(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000602D1CS', m1_daily_df['P'], 0)
m1_daily_df['TVS_NOW'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001700CS', m1_daily_df['P'], 0)
# m1_daily_df['TVS_CMDU'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001004CS', m1_daily_df['P'], 0)

# m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_daily_df['DoD'] = m1_daily_df['TOTAL'] - m1_daily_df['TOTAL'].shift()
# m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_daily_df['DoD'] = m1_daily_df['TOTAL'] - m1_daily_df['TOTAL'].shift()
m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_daily_df_display = m1_daily_df.copy()
mod_col_list = m1_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_daily_df_display[col] = m1_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202506,20250625,2025-06-26 01:10:35,"-4,321,668",0,0,0,0,0,0,0,0,0,0,0
1,202506,20250624,2025-06-26 01:10:35,"-5,679,647","4,321,668","3,628,486","693,182",0,0,0,"2,861,880","2,256,131","605,749","1,015,419",0
2,202506,20250623,2025-06-26 01:10:35,"-6,079,299","10,001,315","2,811,876","7,189,438",0,0,"6,435,059","2,161,382","1,557,871","603,511","1,014,296","10,018"
3,202506,20250622,2025-06-26 01:10:35,"-187,847","16,080,613","8,715,476","7,365,137","12,517,777","5,819,300","6,698,477","2,611,180","1,953,409","657,771","899,900","12,502"
4,202506,20250621,2025-06-26 01:10:35,"-4,679,260","16,268,460","8,882,741","7,385,719","12,719,194","5,903,087","6,816,107","2,415,353","1,866,206","549,147","1,024,445","13,749"
5,202506,20250620,2025-06-26 01:10:35,"5,094,268","20,947,720","9,807,594","11,140,127","13,274,976","6,190,642","7,084,334","2,336,786","1,729,879","606,907","978,775","12,441"
6,202506,20250619,2025-06-26 01:10:35,"617,404","15,853,452","8,626,339","7,227,113","12,165,071","5,814,355","6,350,716","2,243,892","1,584,016","659,876","940,379","6,589"
7,202506,20250618,2025-06-26 01:10:35,"2,919","15,236,049","8,479,970","6,756,079","11,660,576","5,695,686","5,964,890","2,252,910","1,534,633","718,277","1,003,282","6,664"
8,202506,20250617,2025-06-26 01:10:35,"153,677","15,233,129","8,015,610","7,217,519","11,169,199","5,020,630","6,148,568","2,734,151","1,738,299","995,852","1,042,340","7,927"
9,202506,20250616,2025-06-26 01:10:35,"664,526","15,079,452","8,182,033","6,897,419","11,012,365","5,084,632","5,927,733","2,205,532","1,493,579","711,953","1,097,280","15,525"


## Overview(Geo) by Period

In [9]:
''' Inflow M1(Geo Channel) Yearly '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_geo_yearly_df = m1_geo_yearly_df.loc[m1_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_yearly_df['TOTAL'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOTAL_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOTAL_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B2R010500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB2R010500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB2R010500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B2R020500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB2R020500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB2R020500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOL_CONN'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000601CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_CONN(Install)'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_DSV'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000602CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_DSV(Install)'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TVS_NOW'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB4R001700CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TVS_CMDU'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB4R001004CG', m1_geo_yearly_df['P'], 0)

# m1_geo_yearly_df = m1_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_yearly_df['YoY'] = m1_geo_yearly_df['TOTAL'] - m1_geo_yearly_df['TOTAL'].shift()
# m1_geo_yearly_df = m1_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# m1_geo_yearly_df = m1_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_yearly_df = m1_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_yearly_df['YoY'] = m1_geo_yearly_df['TOTAL'] - m1_geo_yearly_df['TOTAL'].shift()
m1_geo_yearly_df = m1_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
m1_geo_yearly_df = m1_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_yearly_df_display = m1_geo_yearly_df.copy()
m1_geo_yearly_df_display['ACTUAL_AS_OF'] = m1_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_yearly_df_display[col] = m1_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025,2025-06-26 01:10:35,20250624,0,"2,525,899,704","1,445,050,730","1,080,848,975","2,140,409,725","1,109,910,964","1,035,771,766","234,090,755","191,364,123","42,726,632","123,500,001","2,187,038"


In [10]:
''' Inflow M1(Geo Channel) Quarterly '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_geo_quarterly_df = m1_geo_quarterly_df.loc[m1_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_quarterly_df['TOTAL'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOTAL_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOTAL_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B2R010500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB2R010500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB2R010500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B2R020500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB2R020500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB2R020500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOL_CONN'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000601CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_CONN(Install)'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_DSV'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000602CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_DSV(Install)'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TVS_NOW'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB4R001700CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TVS_CMDU'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB4R001004CG', m1_geo_quarterly_df['P'], 0)

# m1_geo_quarterly_df = m1_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_quarterly_df['QoQ'] = m1_geo_quarterly_df['TOTAL'] - m1_geo_quarterly_df['TOTAL'].shift()
# m1_geo_quarterly_df = m1_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# m1_geo_quarterly_df = m1_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_quarterly_df = m1_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_quarterly_df['QoQ'] = m1_geo_quarterly_df['TOTAL'] - m1_geo_quarterly_df['TOTAL'].shift()
m1_geo_quarterly_df = m1_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
m1_geo_quarterly_df = m1_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_quarterly_df_display = m1_geo_quarterly_df.copy()
m1_geo_quarterly_df_display['ACTUAL_AS_OF'] = m1_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_quarterly_df_display[col] = m1_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,20251,2025-06-26 01:10:35,20250331,0,"1,378,838,395","790,266,697","588,571,699","1,179,496,714","614,011,326","565,485,388","124,890,149","102,959,079","21,931,070","62,397,203","1,111,862"
1,20252,2025-06-26 01:10:35,20250624,"-231,777,086","1,147,061,309","654,784,033","492,277,276","960,913,011","495,899,638","470,286,378","109,200,606","88,405,044","20,795,562","61,102,798","1,075,176"


In [11]:
''' Inflow M1(Geo Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
m1_geo_monthly_df = m1_geo_monthly_df.loc[m1_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_monthly_df['TOTAL'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOTAL_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOTAL_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B2R010500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB2R010500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB2R010500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B2R020500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB2R020500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB2R020500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOL_CONN'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000601CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_CONN(Install)'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_DSV'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000602CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_DSV(Install)'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TVS_NOW'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB4R001700CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TVS_CMDU'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB4R001004CG', m1_geo_monthly_df['P'], 0)

# m1_geo_monthly_df = m1_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_monthly_df['MoM'] = m1_geo_monthly_df['TOTAL'] - m1_geo_monthly_df['TOTAL'].shift()
# m1_geo_monthly_df = m1_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# m1_geo_monthly_df = m1_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_monthly_df = m1_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_monthly_df['MoM'] = m1_geo_monthly_df['TOTAL'] - m1_geo_monthly_df['TOTAL'].shift()
m1_geo_monthly_df = m1_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
m1_geo_monthly_df = m1_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_monthly_df_display = m1_geo_monthly_df.copy()
m1_geo_monthly_df_display['ACTUAL_AS_OF'] = m1_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_monthly_df_display[col] = m1_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202501,2025-06-26 01:10:35,20250131,0,"491,850,599","278,217,598","213,633,001","429,927,804","223,753,635","206,174,169","38,555,159","31,477,331","7,077,828","19,471,134","376,664"
1,202502,2025-06-26 01:10:35,20250228,"-52,938,633","438,911,966","247,017,359","191,894,607","375,063,654","190,565,264","184,498,390","39,986,117","32,971,912","7,014,205","19,961,102","264,749"
2,202503,2025-06-26 01:10:35,20250331,"9,163,864","448,075,830","265,031,740","183,044,091","374,505,256","199,692,427","174,812,829","46,348,874","38,509,836","7,839,038","22,964,967","470,449"
3,202504,2025-06-26 01:10:35,20250430,"-18,662,210","429,413,620","251,562,458","177,851,162","359,407,356","189,643,908","169,763,448","45,183,640","37,483,652","7,699,989","20,901,734","441,246"
4,202505,2025-06-26 01:10:35,20250531,"1,471,543","430,885,163","244,187,397","186,697,766","367,391,197","188,609,826","178,781,371","36,970,047","29,453,492","7,516,555","22,391,575","400,240"
5,202506,2025-06-26 01:10:35,20250624,"-144,122,637","286,762,526","159,034,178","127,728,348","234,114,457","117,645,904","121,741,559","27,046,918","21,467,900","5,579,018","17,809,489","233,691"


In [12]:
''' Inflow M1(Geo Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_geo_weekly_df = m1_geo_weekly_df.loc[m1_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_weekly_df['TOTAL'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOTAL_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOTAL_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B2R010500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB2R010500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB2R010500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B2R020500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB2R020500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB2R020500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOL_CONN'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000601CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_CONN(Install)'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_DSV'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000602CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_DSV(Install)'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TVS_NOW'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB4R001700CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TVS_CMDU'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB4R001004CG', m1_geo_weekly_df['P'], 0)

# m1_geo_weekly_df = m1_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_weekly_df['WoW'] = m1_geo_weekly_df['TOTAL'] - m1_geo_weekly_df['TOTAL'].shift()
# m1_geo_weekly_df = m1_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# m1_geo_weekly_df = m1_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_weekly_df = m1_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_weekly_df['WoW'] = m1_geo_weekly_df['TOTAL'] - m1_geo_weekly_df['TOTAL'].shift()
m1_geo_weekly_df = m1_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
m1_geo_weekly_df = m1_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_weekly_df_display = m1_geo_weekly_df.copy()
m1_geo_weekly_df_display['ACTUAL_AS_OF'] = m1_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_weekly_df_display[col] = m1_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025018,2025-06-26 01:10:35,20250504,0,"55,839,974","31,988,015","23,851,959","46,711,093","24,152,685","22,558,408","6,334,713","5,077,813","1,256,900","2,506,557","63,822"
1,2025019,2025-06-26 01:10:35,20250511,"40,514,585","96,354,560","57,007,906","39,346,653","80,647,501","43,125,564","37,521,938","9,377,275","7,638,790","1,738,485","5,640,699","72,810"
2,2025020,2025-06-26 01:10:35,20250518,"1,514,072","97,868,631","58,291,069","39,577,562","83,327,904","45,506,441","37,821,463","8,481,078","6,811,667","1,669,411","5,158,992","55,989"
3,2025021,2025-06-26 01:10:35,20250525,"-3,272,368","94,596,263","50,417,676","44,178,588","81,567,536","39,068,345","42,499,192","6,977,220","5,425,771","1,551,449","4,970,669","98,954"
4,2025022,2025-06-26 01:10:35,20250601,"4,865,338","99,461,601","53,202,317","46,259,284","86,226,329","41,749,391","44,476,938","7,165,152","5,476,523","1,688,629","4,793,804","122,067"
5,2025023,2025-06-26 01:10:35,20250608,"-11,502,544","87,959,057","47,574,531","40,384,526","73,731,454","35,316,994","38,414,460","8,287,233","6,460,807","1,826,426","5,245,506","78,739"
6,2025024,2025-06-26 01:10:35,20250615,"-1,761,438","86,197,620","49,951,047","36,246,573","72,874,575","38,197,545","34,677,031","7,849,733","6,360,113","1,489,620","4,921,472","56,134"
7,2025025,2025-06-26 01:10:35,20250622,"3,923,598","90,121,217","51,295,472","38,825,745","76,419,262","39,138,766","37,280,496","7,397,966","5,944,411","1,453,555","5,413,378","75,397"
8,2025026,2025-06-26 01:10:35,20250624,"-80,872,452","9,248,765","3,493,542","5,755,223",0,0,"5,273,005","2,146,596","1,725,498","421,098","1,549,987","10,018"


In [13]:
''' Inflow M1(Geo Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_geo_daily_df = m1_geo_daily_df.loc[m1_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_daily_df['TOTAL'] = np.where(m1_geo_daily_df['METRIC_CD']=='B0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE'] = np.where(m1_geo_daily_df['METRIC_CD']=='B1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R010500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R020500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R020500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R020500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOL_CONN'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_CONN(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_DSV'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000602CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_DSV(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TVS_NOW'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001700CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TVS_CMDU'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001004CG', m1_geo_daily_df['P'], 0)

# m1_geo_daily_df = m1_geo_daily_df.groupby('TM_KEY_MTH', 'TM_KEY_DAY').agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_daily_df['DoD'] = m1_geo_daily_df['TOTAL'] - m1_geo_daily_df['TOTAL'].shift()
# m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_daily_df = m1_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_daily_df['DoD'] = m1_geo_daily_df['TOTAL'] - m1_geo_daily_df['TOTAL'].shift()
m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_daily_df_display = m1_geo_daily_df.copy()
mod_col_list = m1_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_daily_df_display[col] = m1_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202506,20250625,2025-06-26 01:10:35,"-2,058,403",0,0,0,0,0,0,0,0,0,0,0
1,202506,20250624,2025-06-26 01:10:35,"-5,131,958","2,058,403","1,797,411","260,992",0,0,0,"1,144,770","928,344","216,426","760,786",0
2,202506,20250623,2025-06-26 01:10:35,"-5,928,102","7,190,362","1,696,130","5,494,231",0,0,"5,273,005","1,001,826","797,154","204,672","789,201","10,018"
3,202506,20250622,2025-06-26 01:10:35,"-284,817","13,118,464","7,246,037","5,872,426","11,365,492","5,734,324","5,631,168","1,059,716","827,347","232,369","651,916","12,502"
4,202506,20250621,2025-06-26 01:10:35,"-718,868","13,403,280","7,512,784","5,890,496","11,489,599","5,846,689","5,642,910","1,104,241","875,989","228,252","747,458","13,749"
5,202506,20250620,2025-06-26 01:10:35,"1,277,532","14,122,148","8,119,856","6,002,292","11,953,552","6,143,605","5,809,947","1,061,225","883,309","177,916","766,229","12,441"
6,202506,20250619,2025-06-26 01:10:35,"382,385","12,844,616","7,438,352","5,406,264","10,965,305","5,760,944","5,204,361","1,022,959","832,883","190,076","740,694","6,589"
7,202506,20250618,2025-06-26 01:10:35,"134,076","12,462,231","7,337,251","5,124,980","10,569,985","5,641,556","4,928,429","1,004,182","823,180","181,002","799,875","6,664"
8,202506,20250617,2025-06-26 01:10:35,"485,831","12,328,155","6,854,878","5,473,277","10,170,597","4,968,650","5,201,948","1,208,512","951,464","257,048","837,793","7,927"
9,202506,20250616,2025-06-26 01:10:35,"-106,252","11,842,324","6,786,314","5,056,010","9,904,731","5,042,998","4,861,733","937,132","750,240","186,892","869,413","15,525"


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202506


### All Service

In [16]:
''' All Service (All Channel) '''

v_product_grp = 'All Services'

all_service_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_df = all_service_df.loc[~all_service_df['METRIC_NAME'].str.contains('GEO')]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,All Services,B0R00010001CS,Total Inflow M1,2025-06-26 01:10:35,"360,581,626","360,581,626","299,308,135","299,389,527","293,372,086"
1,202506,All Services,DB0R00010001CS,Total Inflow M1 : DTAC,2025-06-26 01:10:35,"167,224,104","167,224,104","158,674,191","158,674,191","155,195,962"
2,202506,All Services,TB0R00010001CS,Total Inflow M1 : TRUE,2025-06-26 01:10:35,"193,357,522","193,357,522","140,633,944","140,715,336","138,176,124"


In [17]:
''' All Service (Geo Channel) '''

v_product_grp = 'All Services'

all_service_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['METRIC_NAME'].str.contains('GEO')]
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_geo_df = all_service_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
all_service_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,All Services,B0R00010001CG,Total Inflow M1 - GEO Channel,2025-06-26 01:10:35,0,"286,762,526","254,263,335","254,344,727","248,521,914"
1,202506,All Services,DB0R00010001CG,Total Inflow M1 : DTAC - GEO Channel,2025-06-26 01:10:35,0,"127,728,348","127,013,003","127,013,003","123,729,231"
2,202506,All Services,TB0R00010001CG,Total Inflow M1 : TRUE - GEO Channel,2025-06-26 01:10:35,0,"159,034,178","127,250,332","127,331,725","124,792,683"


### Prepaid

In [18]:
''' Prepaid (All Channel) '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[~prepaid_df['METRIC_NAME'].str.contains('GEO')]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Prepaid,B1R000900CS,Prepaid Inflow M1,2025-06-26 01:10:35,"257,444,063","257,444,063","222,460,403","222,460,403","216,772,167"
1,202506,Prepaid,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-06-26 01:10:35,"144,821,125","144,821,125","142,753,949","142,753,949","139,275,720"
2,202506,Prepaid,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-06-26 01:10:35,"119,057,998","119,057,998","85,890,310","85,890,310","83,553,769"


In [19]:
''' Prepaid (Geo Channel) '''

v_product_grp = 'Prepaid'

prepaid_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['METRIC_NAME'].str.contains('GEO')]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_geo_df = prepaid_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Prepaid,B1R000900CG,Prepaid Inflow M1 - GEO Channel,2025-06-26 01:10:35,0,"234,114,457","201,711,939","201,711,939","196,209,094"
1,202506,Prepaid,DB1R000900CG,Prepaid Inflow M1 : DTAC - GEO Channel,2025-06-26 01:10:35,0,"121,741,559","121,036,050","121,036,050","117,752,278"
2,202506,Prepaid,TB1R000900CG,Prepaid Inflow M1 : TMH - GEO Channel,2025-06-26 01:10:35,0,"117,645,904","85,879,667","85,879,667","83,543,126"


### Postpaid

In [20]:
''' Postpaid B2C (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[~postpaid_b2c_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Postpaid,B2R010500CS,Postpaid Inflow M1 B2C,2025-06-26 01:10:35,"60,535,332","60,535,332","49,441,882","49,441,882","49,426,778"
1,202506,Postpaid,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-06-26 01:10:35,"14,891,852","14,891,852","14,781,818","14,781,818","14,781,818"
2,202506,Postpaid,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-06-26 01:10:35,"45,643,480","45,643,480","34,660,064","34,660,064","34,660,064"


In [21]:
''' Postpaid B2C (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Postpaid,B2R010500CG,Postpaid Inflow M1 B2C - GEO Channel,2025-06-26 01:10:35,0,"27,046,918","27,036,683","26,958,346","26,247,096"
1,202506,Postpaid,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,2025-06-26 01:10:35,0,"5,579,018","5,569,182","5,569,182","5,569,182"
2,202506,Postpaid,TB2R010500CG,Postpaid Inflow M1 B2C : TMH - GEO Channel,2025-06-26 01:10:35,0,"21,467,900","21,467,501","21,467,501","21,467,671"


In [22]:
''' Postpaid B2B (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[~postpaid_b2b_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Postpaid,B2R020500CS,Postpaid Inflow M1 B2B,2025-06-26 01:10:35,"11,791,424","11,791,424","2,922,727","2,672,878","2,560,598"
1,202506,Postpaid,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-06-26 01:10:35,"7,511,127","7,511,127","1,138,424","1,138,424","1,138,424"
2,202506,Postpaid,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-06-26 01:10:35,"4,280,297","4,280,297","1,803,288","1,803,410","1,803,410"


In [23]:
''' Postpaid B2B (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Postpaid,B2R020500CG,Postpaid Inflow M1 B2B - GEO Channel,2025-06-26 01:10:35,0,"2,208,314","2,185,835","1,675,745","1,756,584"
1,202506,Postpaid,DB2R020500CG,Postpaid Inflow M1 B2B : DTAC - GEO Channel,2025-06-26 01:10:35,0,"407,771","407,771","407,771","407,771"
2,202506,Postpaid,TB2R020500CG,Postpaid Inflow M1 B2B : TMH - GEO Channel,2025-06-26 01:10:35,0,"1,800,543","1,800,543","1,800,665","1,800,665"


### TOL

In [24]:
''' TOL Connected : Consumer (All Channel) '''

v_product_grp = 'TOL'

tol_conn_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_df = tol_conn_con_df.loc[~tol_conn_con_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_df = tol_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-06-26 01:10:35,"22,776,269","22,776,269","17,809,522","17,809,522","17,601,140"
1,202506,TOL,TB3R000601D1CS,TOL Inflow M1 Connected : Consumer (Install Lo...,2025-06-26 01:10:35,"22,776,269","22,776,269","22,702,130","22,702,130","22,714,500"


In [25]:
''' TOL Connected : Consumer (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_con_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_geo_df = tol_conn_con_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3R000601CG,TOL Inflow M1 Connected : Consumer - GEO Channel,2025-06-26 01:10:35,0,"17,809,489","17,808,990","17,808,990","17,600,608"
1,202506,TOL,TB3R000601D1CG,TOL Inflow M1 Connected : Consumer - GEO Chann...,2025-06-26 01:10:35,0,"17,809,489","17,808,790","17,808,790","17,821,160"


In [26]:
''' TOL Connected : DataService (All Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[~tol_conn_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_df = tol_conn_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-06-26 01:10:35,"1,206,149","1,206,149","70,671","70,671","70,671"
1,202506,TOL,TB3R000602D1CS,TOL Inflow M1 Connected : DataService (Install...,2025-06-26 01:10:35,"1,206,149","1,206,149","70,671","70,671","70,671"


In [27]:
''' TOL Connected : DataService (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[~tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-06-26 01:10:35,"1,206,149","1,206,149","70,671","70,671","70,671"
1,202506,TOL,TB3R000602D1CS,TOL Inflow M1 Connected : DataService (Install...,2025-06-26 01:10:35,"1,206,149","1,206,149","70,671","70,671","70,671"


### TVS

In [28]:
''' TVS Now '''

v_product_grp = 'TVS'

tvs_now_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_now_df = tvs_now_df.loc[tvs_now_df['METRIC_NAME'].str.contains('Now')]
tvs_now_df = tvs_now_df.loc[tvs_now_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_now_df = tvs_now_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_now_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TVS,TB4R001700CS,TVS Now Inflow M1,2025-06-26 01:10:35,"233,913","233,913","216,979","298,250","303,961"
1,202506,TVS,TB4R001700CG,TVS Now Inflow M1 - GEO Channel,2025-06-26 01:10:35,0,"233,691","216,979","298,250","303,961"


In [29]:
''' TVS CMDU '''

v_product_grp = 'TVS'

tvs_cmdu_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['METRIC_NAME'].str.contains('CMDU')]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_cmdu_df = tvs_cmdu_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_cmdu_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TVS,TB4R001004CS,TVS CMDU Inflow M1,2025-06-26 01:10:35,"159,416","159,416","183,110","183,110","183,110"
1,202506,TVS,TB4R001004CG,TVS CMDU Inflow M1 - GEO Channel,2025-06-26 01:10:35,0,"5,981","5,981","5,981","5,981"


## ** Special Metric (All & Geo)

### Inflow (All Channel)
    B0R00010001CS   Total Inflow M1
    TB0R00010001CS  Total Inflow M1 : TRUE
    DB0R00010001CS  Total Inflow M1 : DTAC

In [30]:
''' B0R00010001CS : Total Inflow M1 '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CS', 'DB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'DB2R010500CS', 'DB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS')

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1

    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService

    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

total_m1_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_df = total_m1_df.loc[total_m1_df['METRIC_CD'].isin(v_metric_list)]

total_m1_df['TOTAL'] = np.where(total_m1_df['METRIC_CD']=='B0R00010001CS', total_m1_df['P'], 0)

total_m1_df['PRE_T'] = np.where(total_m1_df['METRIC_CD']=='TB1R000900CS', total_m1_df['P'], 0)
total_m1_df['PRE_D'] = np.where(total_m1_df['METRIC_CD']=='DB1R000900CS', total_m1_df['P'], 0)

total_m1_df['POST_B2C_T'] = np.where(total_m1_df['METRIC_CD']=='TB2R010500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2C_D'] = np.where(total_m1_df['METRIC_CD']=='DB2R010500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2B_T'] = np.where(total_m1_df['METRIC_CD']=='TB2R020500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2B_D'] = np.where(total_m1_df['METRIC_CD']=='DB2R020500CS', total_m1_df['P'], 0)

total_m1_df['TOL_CON'] = np.where(total_m1_df['METRIC_CD']=='TB3R000601CS', total_m1_df['P'], 0)
total_m1_df['TOL_DSV'] = np.where(total_m1_df['METRIC_CD']=='TB3R000602CS', total_m1_df['P'], 0)

total_m1_df['TVS_NOW'] = np.where(total_m1_df['METRIC_CD']=='TB4R001700CS', total_m1_df['P'], 0)
total_m1_df['TVS_CMDU'] = np.where(total_m1_df['METRIC_CD']=='TB4R001004CS', total_m1_df['P'], 0)

total_m1_df['CHK_SUM'] = np.where(total_m1_df['METRIC_CD'].isin(['TB1R000900CS', 'DB1R000900CS', 'TB2R010500CS', 'DB2R010500CS', 'TB2R020500CS', 'DB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS']), total_m1_df['P'], 0)

total_m1_df = total_m1_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_df['CHK_DIFF'] = total_m1_df['TOTAL'] - total_m1_df['CHK_SUM']
total_m1_df = total_m1_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_df = total_m1_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_df[col] = total_m1_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,POST_B2B_T,POST_B2B_D,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-06-26 01:10:35,0,"639,880,599","639,880,599","225,749,189","282,966,592","61,334,089","18,126,426","4,946,013","18,864,945","25,646,982","829,033","382,335","1,034,995"
1,202502,2025-06-26 01:10:35,0,"575,180,401","575,180,401","192,368,192","249,735,487","63,185,463","17,947,144","4,637,783","19,426,521","25,046,745","1,648,415","267,183","917,468"
2,202503,2025-06-26 01:10:35,0,"563,517,561","563,517,561","201,578,035","226,924,550","70,909,630","17,637,272","5,801,792","8,532,545","28,791,201","2,517,766","470,671","354,099"
3,202504,2025-06-26 01:10:35,-0,"534,036,147","534,036,147","191,822,637","204,019,741","69,118,915","18,670,677","5,402,830","15,549,925","26,236,110","2,340,768","441,580","432,964"
4,202505,2025-06-26 01:10:35,0,"532,503,207","532,503,207","190,514,414","209,649,746","59,723,743","19,443,366","6,239,659","16,299,019","28,292,314","1,722,859","401,782","216,305"
5,202506,2025-06-26 01:10:35,0,"360,581,626","360,581,626","119,057,998","144,821,125","45,643,480","14,891,852","4,280,297","7,511,127","22,776,269","1,206,149","233,913","159,416"


In [31]:
''' TB0R00010001CS : Total Inflow M1 : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS')

v_metric_list = [
    'TB0R00010001CS' #Total Inflow M1 : TRUE

    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH

    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService

    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

total_m1_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_true_df = total_m1_true_df.loc[total_m1_true_df['METRIC_CD'].isin(v_metric_list)]

total_m1_true_df['TOTAL_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB0R00010001CS', total_m1_true_df['P'], 0)

total_m1_true_df['PRE_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB1R000900CS', total_m1_true_df['P'], 0)

total_m1_true_df['POST_B2C_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB2R010500CS', total_m1_true_df['P'], 0)
total_m1_true_df['POST_B2B_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB2R020500CS', total_m1_true_df['P'], 0)

total_m1_true_df['TOL_CON'] = np.where(total_m1_true_df['METRIC_CD']=='TB3R000601CS', total_m1_true_df['P'], 0)
total_m1_true_df['TOL_DSV'] = np.where(total_m1_true_df['METRIC_CD']=='TB3R000602CS', total_m1_true_df['P'], 0)

total_m1_true_df['TVS_NOW'] = np.where(total_m1_true_df['METRIC_CD']=='TB4R001700CS', total_m1_true_df['P'], 0)
total_m1_true_df['TVS_CMDU'] = np.where(total_m1_true_df['METRIC_CD']=='TB4R001004CS', total_m1_true_df['P'], 0)

total_m1_true_df['CHK_SUM'] = np.where(total_m1_true_df['METRIC_CD'].isin(['TB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS']), total_m1_true_df['P'], 0)

total_m1_true_df = total_m1_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'POST_B2B_T':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_true_df['CHK_DIFF'] = total_m1_true_df['TOTAL_T'] - total_m1_true_df['CHK_SUM']
total_m1_true_df = total_m1_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_true_df = total_m1_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'POST_B2B_T', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_true_df[col] = total_m1_true_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,POST_B2B_T,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-06-26 01:10:35,0,"319,922,636","319,922,636","225,749,189","61,334,089","4,946,013","25,646,982","829,033","382,335","1,034,995"
1,202502,2025-06-26 01:10:35,0,"288,071,249","288,071,249","192,368,192","63,185,463","4,637,783","25,046,745","1,648,415","267,183","917,468"
2,202503,2025-06-26 01:10:35,0,"310,423,195","310,423,195","201,578,035","70,909,630","5,801,792","28,791,201","2,517,766","470,671","354,099"
3,202504,2025-06-26 01:10:35,0,"295,795,804","295,795,804","191,822,637","69,118,915","5,402,830","26,236,110","2,340,768","441,580","432,964"
4,202505,2025-06-26 01:10:35,0,"287,111,076","287,111,076","190,514,414","59,723,743","6,239,659","28,292,314","1,722,859","401,782","216,305"
5,202506,2025-06-26 01:10:35,-0,"193,357,522","193,357,522","119,057,998","45,643,480","4,280,297","22,776,269","1,206,149","233,913","159,416"


In [32]:
''' DB0R00010001CS : Total Inflow M1 : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000900CS', 'DB2R010500CS', 'DB2R020500CS')

v_metric_list = [
    'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    ]

total_m1_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_dtac_df = total_m1_dtac_df.loc[total_m1_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_m1_dtac_df['TOTAL_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB0R00010001CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['PRE_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB1R000900CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['POST_B2C_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB2R010500CS', total_m1_dtac_df['P'], 0)
total_m1_dtac_df['POST_B2B_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB2R020500CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['CHK_SUM'] = np.where(total_m1_dtac_df['METRIC_CD'].isin(['DB1R000900CS', 'DB2R010500CS', 'DB2R020500CS']), total_m1_dtac_df['P'], 0)

total_m1_dtac_df = total_m1_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'POST_B2B_D':'sum', 'CHK_SUM':'sum'})
total_m1_dtac_df['CHK_DIFF'] = total_m1_dtac_df['TOTAL_D'] - total_m1_dtac_df['CHK_SUM']
total_m1_dtac_df = total_m1_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_dtac_df = total_m1_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D', 'POST_B2B_D']]

mod_col_list = total_m1_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_dtac_df[col] = total_m1_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D,POST_B2B_D
0,202501,2025-06-26 01:10:35,0,"319,957,964","319,957,964","282,966,592","18,126,426","18,864,945"
1,202502,2025-06-26 01:10:35,0,"287,109,152","287,109,152","249,735,487","17,947,144","19,426,521"
2,202503,2025-06-26 01:10:35,0,"253,094,366","253,094,366","226,924,550","17,637,272","8,532,545"
3,202504,2025-06-26 01:10:35,0,"238,240,343","238,240,343","204,019,741","18,670,677","15,549,925"
4,202505,2025-06-26 01:10:35,0,"245,392,131","245,392,131","209,649,746","19,443,366","16,299,019"
5,202506,2025-06-26 01:10:35,-0,"167,224,104","167,224,104","144,821,125","14,891,852","7,511,127"


### Inflow (Geo Channel)
    B0R00010001CG   Total Inflow M1 - GEO Channel
    TB0R00010001CG  Total Inflow M1 : TRUE - GEO Channel
    DB0R00010001CG  Total Inflow M1 : DTAC - GEO Channel

In [33]:
''' B0R00010001CG : Total Inflow M1 - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CG', 'DB1R000900CG', 'TB2R010500CG', 'DB2R010500CG', 'TB2R020500CG', 'DB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG')

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    
    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

total_m1_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_geo_df = total_m1_geo_df.loc[total_m1_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_geo_df['TOTAL'] = np.where(total_m1_geo_df['METRIC_CD']=='B0R00010001CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['PRE_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB1R000900CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['PRE_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB1R000900CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['POST_B2C_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB2R010500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2C_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB2R010500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2B_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB2R020500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2B_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB2R020500CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['TOL_CON'] = np.where(total_m1_geo_df['METRIC_CD']=='TB3R000601CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['TOL_DSV'] = np.where(total_m1_geo_df['METRIC_CD']=='TB3R000602CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['TVS_NOW'] = np.where(total_m1_geo_df['METRIC_CD']=='TB4R001700CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['TVS_CMDU'] = np.where(total_m1_geo_df['METRIC_CD']=='TB4R001004CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['CHK_SUM'] = np.where(total_m1_geo_df['METRIC_CD'].isin(['TB1R000900CG', 'DB1R000900CG', 'TB2R010500CG', 'DB2R010500CG', 'TB2R020500CG', 'DB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG']), total_m1_geo_df['P'], 0)

total_m1_geo_df = total_m1_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_geo_df['CHK_DIFF'] = total_m1_geo_df['TOTAL'] - total_m1_geo_df['CHK_SUM']
total_m1_geo_df = total_m1_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_geo_df = total_m1_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_geo_df[col] = total_m1_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,POST_B2B_T,POST_B2B_D,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-06-26 01:10:35,0,"491,850,599","491,850,599","223,753,635","206,174,169","31,477,331","7,077,828","2,357,448","381,005","19,471,134","397,930","376,664","383,456"
1,202502,2025-06-26 01:10:35,-0,"438,911,966","438,911,966","190,565,264","184,498,390","32,971,912","7,014,205","2,468,889","382,012","19,961,102","423,582","264,749","361,861"
2,202503,2025-06-26 01:10:35,0,"448,075,830","448,075,830","199,692,427","174,812,829","38,509,836","7,839,038","3,206,656","392,224","22,964,967","133,660","470,449","53,744"
3,202504,2025-06-26 01:10:35,-0,"429,413,620","429,413,620","189,643,908","169,763,448","37,483,652","7,699,989","2,886,311","387,725","20,901,734","134,885","441,246","70,723"
4,202505,2025-06-26 01:10:35,0,"430,885,163","430,885,163","188,609,826","178,781,371","29,453,492","7,516,555","3,110,301","399,840","22,391,575","221,363","400,240",600
5,202506,2025-06-26 01:10:35,0,"286,762,526","286,762,526","117,645,904","121,741,559","21,467,900","5,579,018","1,800,543","407,771","17,809,489","70,671","233,691","5,981"


In [34]:
''' TB0R00010001CG : Total Inflow M1 : TRUE - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CG', 'TB2R010500CG', 'TB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG')

v_metric_list = [
    'TB0R00010001CG' #Total Inflow M1 - GEO Channel
    
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    
    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

total_m1_true_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_true_geo_df = total_m1_true_geo_df.loc[total_m1_true_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_true_geo_df['TOTAL_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB0R00010001CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['PRE_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB1R000900CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['POST_B2C_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB2R010500CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['POST_B2B_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB2R020500CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['TOL_CON'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB3R000601CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['TOL_DSV'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB3R000602CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['TVS_NOW'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB4R001700CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['TVS_CMDU'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB4R001004CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['CHK_SUM'] = np.where(total_m1_true_geo_df['METRIC_CD'].isin(['TB1R000900CG', 'TB2R010500CG', 'TB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG']), total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df = total_m1_true_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'POST_B2B_T':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_true_geo_df['CHK_DIFF'] = total_m1_true_geo_df['TOTAL_T'] - total_m1_true_geo_df['CHK_SUM']
total_m1_true_geo_df = total_m1_true_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_true_geo_df = total_m1_true_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'POST_B2B_T', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_true_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_true_geo_df[col] = total_m1_true_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_true_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,POST_B2B_T,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-06-26 01:10:35,0,"278,217,598","278,217,598","223,753,635","31,477,331","2,357,448","19,471,134","397,930","376,664","383,456"
1,202502,2025-06-26 01:10:35,0,"247,017,359","247,017,359","190,565,264","32,971,912","2,468,889","19,961,102","423,582","264,749","361,861"
2,202503,2025-06-26 01:10:35,0,"265,031,740","265,031,740","199,692,427","38,509,836","3,206,656","22,964,967","133,660","470,449","53,744"
3,202504,2025-06-26 01:10:35,0,"251,562,458","251,562,458","189,643,908","37,483,652","2,886,311","20,901,734","134,885","441,246","70,723"
4,202505,2025-06-26 01:10:35,0,"244,187,397","244,187,397","188,609,826","29,453,492","3,110,301","22,391,575","221,363","400,240",600
5,202506,2025-06-26 01:10:35,0,"159,034,178","159,034,178","117,645,904","21,467,900","1,800,543","17,809,489","70,671","233,691","5,981"


In [35]:
''' DB0R00010001CG : Total Inflow M1 : DTAC - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000900CG', 'DB2R010500CG', 'DB2R020500CG')

v_metric_list = [
    'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel
    
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    ]

total_m1_dtac_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_dtac_geo_df = total_m1_dtac_geo_df.loc[total_m1_dtac_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_dtac_geo_df['TOTAL_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB0R00010001CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['PRE_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB1R000900CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['POST_B2C_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB2R010500CG', total_m1_dtac_geo_df['P'], 0)
total_m1_dtac_geo_df['POST_B2B_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB2R020500CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['CHK_SUM'] = np.where(total_m1_dtac_geo_df['METRIC_CD'].isin(['DB1R000900CG', 'DB2R010500CG', 'DB2R020500CG']), total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df = total_m1_dtac_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'POST_B2B_D':'sum', 'CHK_SUM':'sum'})
total_m1_dtac_geo_df['CHK_DIFF'] = total_m1_dtac_geo_df['TOTAL_D'] - total_m1_dtac_geo_df['CHK_SUM']
total_m1_dtac_geo_df = total_m1_dtac_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_dtac_geo_df = total_m1_dtac_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D', 'POST_B2B_D']]

mod_col_list = total_m1_dtac_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_dtac_geo_df[col] = total_m1_dtac_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_dtac_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D,POST_B2B_D
0,202501,2025-06-26 01:10:35,-0,"213,633,001","213,633,001","206,174,169","7,077,828","381,005"
1,202502,2025-06-26 01:10:35,0,"191,894,607","191,894,607","184,498,390","7,014,205","382,012"
2,202503,2025-06-26 01:10:35,0,"183,044,091","183,044,091","174,812,829","7,839,038","392,224"
3,202504,2025-06-26 01:10:35,0,"177,851,162","177,851,162","169,763,448","7,699,989","387,725"
4,202505,2025-06-26 01:10:35,0,"186,697,766","186,697,766","178,781,371","7,516,555","399,840"
5,202506,2025-06-26 01:10:35,0,"127,728,348","127,728,348","121,741,559","5,579,018","407,771"
